In [1]:
import torch
from torch import nn

import torchvision
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
])

In [3]:
train_data = datasets.FashionMNIST(
    root="train",
    train=True,
    download=True,
    transform=transform,
    target_transform=None
)

test_data = datasets.FashionMNIST(
    root="test",
    train=False,
    download=True,
    transform=transform
)

100%|██████████| 26.4M/26.4M [00:02<00:00, 11.9MB/s]


Extracting train/FashionMNIST/raw/train-images-idx3-ubyte.gz to train/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 204kB/s]


Extracting train/FashionMNIST/raw/train-labels-idx1-ubyte.gz to train/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:01<00:00, 3.32MB/s]


...

Extracting test/FashionMNIST/raw/train-labels-idx1-ubyte.gz to test/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:01<00:00, 3.72MB/s]


Extracting test/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to test/FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 19.2MB/s]

Extracting test/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to test/FashionMNIST/raw



In [4]:
image, label = train_data[0]
print(label)
image.shape

9


torch.Size([1, 28, 28])

In [5]:
class_names = train_data.classes
class_names

['T-shirt/top',
 'Trouser',
 'Pullover',
 'Dress',
 'Coat',
 'Sandal',
 'Shirt',
 'Sneaker',
 'Bag',
 'Ankle boot']

In [6]:
len(train_data.data), len(train_data.targets), len(test_data.data), len(test_data.targets)

(60000, 60000, 10000, 10000)

In [7]:
from torch.utils.data import DataLoader
BATCH_SIZE = 32

train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)

print(len(train_dataloader), BATCH_SIZE)
print(len(test_dataloader), BATCH_SIZE)

1875 32
313 32


In [8]:
class FashionMNISTModel(nn.Module):
    def __init__(self, input_shape, hidden_units, output_shape):
        super().__init__()

        self.input_norm = nn.BatchNorm2d(input_shape)
        self.block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.BatchNorm2d(hidden_units),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.BatchNorm2d(hidden_units),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2)
        )
        self.block_2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units, 3, padding=1),
            nn.BatchNorm2d(hidden_units),
            nn.ReLU(),
            nn.Conv2d(hidden_units, hidden_units, 3, padding=1),
            nn.BatchNorm2d(hidden_units),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units*7*7,
                      out_features=output_shape)
        )

    def forward(self, x: torch.Tensor):
        x = self.input_norm(x)
        x = self.block_1(x)
        x = self.block_2(x)
        x = self.classifier(x)
        return x

In [9]:
torch.manual_seed(42)
model = FashionMNISTModel(input_shape=1,
    hidden_units=10,
    output_shape=len(class_names))
model

FashionMNISTModel(
  (input_norm): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (block_1): Sequential(
    (0): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5

In [10]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.1)

In [11]:
def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer):
    train_loss = 0
    for batch, (X, y) in enumerate(data_loader):

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()
    train_loss /= len(data_loader)
    print(f"Train loss: {train_loss:.5f}")

In [13]:
epochs = 10

In [14]:
for epoch in range(epochs):
  train_step(model, train_dataloader, loss_fn, optimizer)

Train loss: 0.47223
Train loss: 0.34053
Train loss: 0.31292
Train loss: 0.29527
Train loss: 0.28316
Train loss: 0.27277
Train loss: 0.26745
Train loss: 0.26129
Train loss: 0.25542
Train loss: 0.25361


In [15]:
image, label = test_data[0]
image = image.unsqueeze(0)
print(label)
print(class_names[label])

9
Ankle boot


In [16]:
model.eval()
y_pred = model(image)
y_pred = torch.softmax(y_pred, dim=1)
y_pred = torch.argmax(y_pred, dim = 1).item()
print(y_pred)
print(class_names[y_pred])

9
Ankle boot
